# Installation

This module is made to be an overlayer of the Google OR Tools cpsolver.
It does not require any python library other than **ortools**.

Full documention is available at : https://datalab-stsisi.github.io/

# Overview

The module enables you to perform in an easy way some operations on cp models:

* Modelization of multiple objectives in a single model, with sequential or combined optimization
* Modelization of relaxable constraints (constraints that are not mandatory but award bonuses if respected)
* Explanation of infeasibility for infeasible problems
* Local explanation of solution for feasible problems
* Production of natural language explanations
* Easier local optimization (LNS)

## 1- First example - explaining infeasibility

We will create here a sudoku problem that is infeasible.

### 1.1- Create a model

In [16]:
import ortools_explain

from ortools_explain.model import SuperModel
from ortools_explain.solver import SuperSolver
from ortools_explain.status import Status

Create a cp_model by calling SuperModel():

In [17]:
model = SuperModel()

### 1.2- Define variables

Define variables as you normally would, with NewBoolVar or NewIntVar.

In [18]:
size_sudoku = 4
x = {(i, j, v): model.NewBoolVar("x_%d_%d_%d" % (i, j, v))
         for i in range(size_sudoku) for j in range(size_sudoku)
         for v in range(1, size_sudoku + 1)}

### 1.3- Add constraints

Constraints are added with functions Add and AddConstant.

1/ We add the constraint that there must be one value and only one by square. We do not name this constraint because we consider it to be a fundamental part of our problem.

In [19]:
for i in range(size_sudoku):
    for j in range(size_sudoku):
        model.Add(sum(x[i, j, v] for v in range(1, size_sudoku + 1)) == 1)

2/ We add the constraint that there must be one value of each per line. We name these constraints "all_different_line" and we give it the dimension "line". This means that in our explanations for infeasible problems, we want the solver to be able to return which line is part of the problem. We could also have used both line and value as dimensions.

In [20]:
for i in range(size_sudoku):
    for v in range(1, size_sudoku + 1):
        model.Add(sum(x[i, j, v] for j in range(size_sudoku)) == 1, "all_different_line", line=i + 1)

3/ We add the same constraint for columns.

In [22]:
for j in range(size_sudoku):
    for v in range(1, size_sudoku + 1):
        model.Add(sum(x[i, j, v] for i in range(size_sudoku)) == 1, "all_different_column", column=j + 1)

4/ We add the same constraint for squares. Here we choose to use both the id of the square and the value as dimensions.

In [23]:
k = int(sqrt(size_sudoku))
for a in range(k):
    for b in range(k):
        for v in range(1, size_sudoku + 1):
            list_cells = [x[i, j, v] for i in range(a * k, (a + 1) * k)
                          for j in range(b * k, (b + 1) * k)]
            model.Add(sum(x for x in list_cells) == 1, "all_different_square",
                      square=str(a + 1) + "_" + str(b + 1), value=v)

5/ We add some initial values to make the problem infeasible. In OR Tools we would use the NewConstant function to do this. Here we use the AddConstant function instead. We also give these constraints a name and dimensions.

In [26]:
initial_positions = [(0, 0, 1), (0, 1, 1), (2, 0, 3), (3, 0, 3), (3, 1, 3)]
for row, col, value in initial_positions:
    model.AddConstant(x[row, col, value], 1, "initial_pos", line=row+1, column=col+1, value=value)

### 1.4- (Optional) Add natural language explanations

Use AddExplanation to match constraint types with natural language explanations. Explanations should ideally be declared for all combinations of dimensions being present or not, since the module will decide automatically which dimensions are relevant on one given instance.

In [27]:
model.AddExplanation("all_different_line", "All lines must contain each value once and only once",
                         "{line}th line must contain each value once and only once")

model.AddExplanation("all_different_column", "All columns must contain each value once and only once",
                         "{column}th column must contain each value once and only once")

model.AddExplanation("all_different_square", "All squares must contain each value once and only once",
                         "Square {square} must contain each value once and only once",
                            "All squares must contain value {value} once and only once",
                            "Square {square} must contain value {value} once and only once")

model.AddExplanation("initial_pos", "Initial positions", "Initial positions on line {line}",
                         "Initial positions on column {column}", "Initial positions of {value}",
                         "Initial position on square ({line}, {column})", "Initial positions of {value} on line {line}",
                         "Initial positions of {value} on column {column}", "The initial {value} at ({line}, {column})")

### 1.5- (Optional) Print the problem

In [32]:
tab_val = dict()
for i in range(size_sudoku):
    for j in range(size_sudoku):
        tab_val[i, j] = '.'
        for v in range(1, size_sudoku+1):
            if (i, j, v) in initial_positions:
                tab_val[i, j] = v
                break

n = size_sudoku
k = int(sqrt(n))
for i in range(n):
    for j in range(n):
        if tab_val[i, j]:
            print(tab_val[i, j], end=" ")
        else:
            print(".", end=" ")
        if j + 1 < n and (j + 1) % k == 0:
            print("|", end=" ")
    print()
    if i + 1 < n and (i + 1) % k == 0:
        print("- " * (n + 1))

1 1 | . . 
. . | . . 
- - - - - 
3 . | . . 
3 3 | . . 


### 1.6- Try to solve the problem

Create a cp_solver by calling SuperSolver().

**Beware that as opposed to the standard OR Tools module, ortools_explain takes the model as argument when created**

In [33]:
solver = SuperSolver(model)

Call Solve() to get the status of the problem:

In [34]:
solver.Solve() == Status.INFEASIBLE

True

### 1.7- Get explanations

Minimum sets of constraints that make the problem infeasible are called IIS. Call ExplainWhyNoSolution() to get a set of iis. 

Please note that even if parameters are the same, ExplainWhyNoSolution is partly random and may not always return the same set of iis.

In [40]:
solver.ExplainWhyNoSolution()

[[{'sentence': '1th line must contain each value once and only once',
   'type': 'all_different_line',
   'dimensions': {'line': 1}},
  {'sentence': 'The initial 1 at (1, 1)',
   'type': 'initial_pos',
   'dimensions': {'column': 1, 'line': 1, 'value': 1}},
  {'sentence': 'The initial 1 at (1, 2)',
   'type': 'initial_pos',
   'dimensions': {'column': 2, 'line': 1, 'value': 1}}],
 [{'sentence': '4th line must contain each value once and only once',
   'type': 'all_different_line',
   'dimensions': {'line': 4}},
  {'sentence': 'The initial 3 at (4, 1)',
   'type': 'initial_pos',
   'dimensions': {'column': 1, 'line': 4, 'value': 3}},
  {'sentence': 'The initial 3 at (4, 2)',
   'type': 'initial_pos',
   'dimensions': {'column': 2, 'line': 4, 'value': 3}}]]

ExplainWhyNoSolution() has a number of optional parameters that you can use to finetune the set of constraints that is returned.

**find_several_iis** is set to True by default. If False, it will only return one set of constraints that makes the problem infeasible. If True, it will return a set of not-overlapping conflicts that is maximum in size (deactivating all constraints of all these conflicts is sufficient but not necessary to make the problem feasible).

In [39]:
solver.ExplainWhyNoSolution(find_several_iis = False)

[[{'sentence': '1th line must contain each value once and only once',
   'type': 'all_different_line',
   'dimensions': {'line': 1}},
  {'sentence': 'The initial 1 at (1, 1)',
   'type': 'initial_pos',
   'dimensions': {'column': 1, 'line': 1, 'value': 1}},
  {'sentence': 'The initial 1 at (1, 2)',
   'type': 'initial_pos',
   'dimensions': {'column': 2, 'line': 1, 'value': 1}}]]

**method_for_search** must be one of SuperSolver.SUFFICIENT_ASSUMPTION, SuperSolver.QUICK_XPLAIN or SuperSolver.PARALLEL_SEARCH (default - runs the other two methods in two threads and returns the fastest one).

All methods should work on all problems but some are faster than others depending on the problem. However using one or the other method will most likely result in different sets of iis.

In [41]:
solver.ExplainWhyNoSolution(find_several_iis = False, method_for_search = SuperSolver.SUFFICIENT_ASSUMPTION)

[[{'sentence': '1th line must contain each value once and only once',
   'type': 'all_different_line',
   'dimensions': {'line': 1}},
  {'sentence': 'The initial 1 at (1, 1)',
   'type': 'initial_pos',
   'dimensions': {'column': 1, 'line': 1, 'value': 1}},
  {'sentence': 'The initial 1 at (1, 2)',
   'type': 'initial_pos',
   'dimensions': {'column': 2, 'line': 1, 'value': 1}}]]

In [42]:
solver.ExplainWhyNoSolution(find_several_iis = False, method_for_search = SuperSolver.QUICK_XPLAIN)

[[{'sentence': '1th column must contain each value once and only once',
   'type': 'all_different_column',
   'dimensions': {'column': 1}},
  {'sentence': 'The initial 3 at (3, 1)',
   'type': 'initial_pos',
   'dimensions': {'column': 1, 'line': 3, 'value': 3}},
  {'sentence': 'The initial 3 at (4, 1)',
   'type': 'initial_pos',
   'dimensions': {'column': 1, 'line': 4, 'value': 3}}]]

**zoom_level** enables you to balance the precision of the result and its size. If zoom_level is high, the result may be of large size but its members will be precise (they will use most dimensions). If zoom_level is low, the result will be of smaller size but constraints in the result will be less precise. 

In [48]:
solver.ExplainWhyNoSolution(find_several_iis = False, zoom_level = 1)

[[{'sentence': '1th line must contain each value once and only once',
   'type': 'all_different_line',
   'dimensions': {'line': 1}},
  {'sentence': 'Initial positions of 1 on line 1',
   'type': 'initial_pos',
   'dimensions': {'line': 1, 'value': 1}}]]

In [49]:
solver.ExplainWhyNoSolution(find_several_iis = False, zoom_level = 2)

[[{'sentence': '1th line must contain each value once and only once',
   'type': 'all_different_line',
   'dimensions': {'line': 1}},
  {'sentence': 'The initial 1 at (1, 1)',
   'type': 'initial_pos',
   'dimensions': {'column': 1, 'line': 1, 'value': 1}},
  {'sentence': 'The initial 1 at (1, 2)',
   'type': 'initial_pos',
   'dimensions': {'column': 2, 'line': 1, 'value': 1}}]]

## 2- Second example - solving with multiple objectives

Here we will create an empty 9*9 sudoku grid to illustrate optimization.

### 2.1- Create a sudoku grid

We create an empty sudoku grid as we did above, by defining variables and constraints.

In [54]:
model = SuperModel()

size_sudoku = 9
x = {(i, j, v): model.NewBoolVar("x_%d_%d_%d" % (i, j, v))
         for i in range(size_sudoku) for j in range(size_sudoku)
         for v in range(1, size_sudoku + 1)}

for i in range(size_sudoku):
    for j in range(size_sudoku):
        model.Add(sum(x[i, j, v] for v in range(1, size_sudoku + 1)) == 1)
        
for i in range(size_sudoku):
    for v in range(1, size_sudoku + 1):
        model.Add(sum(x[i, j, v] for j in range(size_sudoku)) == 1, "all_different_line", line=i + 1)
        
for j in range(size_sudoku):
    for v in range(1, size_sudoku + 1):
        model.Add(sum(x[i, j, v] for i in range(size_sudoku)) == 1, "all_different_column", column=j + 1)
        
k = int(sqrt(size_sudoku))
for a in range(k):
    for b in range(k):
        for v in range(1, size_sudoku + 1):
            list_cells = [x[i, j, v] for i in range(a * k, (a + 1) * k)
                          for j in range(b * k, (b + 1) * k)]
            model.Add(sum(x for x in list_cells) == 1, "all_different_square",
                      square=str(a + 1) + "_" + str(b + 1), value=v)

### 2.2- Define objectives

Here we decide to add some objectives on this grid. The objectives are the following:

* Top-priority:
    * Put a 1 on the square on all squares of the first diagonal
* Mid-level priority:
    * Maximize the sum of the first diagonal
* Low priority:
    * Minimize the sum of the second diagonal
    * Put a 9 in the top right corner
    
In our module, objectives are added with functions AddRelaxableConstraint, AddMaximumObjective and AddMinimumObjective, along with a priority. The module will consider objectives by increasing order of priority.
    
#### 2.2.1- Defining not mandatory constraints

To implement the top-priority constraint, we use the AddRelaxable function. This function allows you to add a constraint that is not mandatory but awards points if respected.

In [55]:
for i in range(size_sudoku):
    model.AddRelaxableConstraint(x[i, i, 1] == 1, idx="Numbers of 1 on first diagonal", coef=1, priority=1)

#### 2.2.2- Defining partial objectives

To implement the mid-level priority, we use the AddMaximumObjective function.

In [56]:
model.AddMaximumObjective(sum(v * x[i, i, v] for i in range(size_sudoku) for v in range(1, size_sudoku + 1)), priority=2,
                              idx='Maximize sum of first diagonal')

#### 2.2.3- Balancing objectives

You may want to define several objectives with the same priority, because they are conflicting and you want to find a solution that balances them. In this case, you could use coefficients to give additional importance to one of the conflicts.

In [57]:
coef_for_sum = 3
coef_for_top_right_corner = 40

model.AddMinimumObjective(coef_for_sum * sum(v * x[i, size_sudoku - 1 - i, v] for i in range(size_sudoku) for v in range(1, size_sudoku + 1)),
                              priority=3, idx='Minimize sum of second diagonal')
model.AddRelaxableConstraint(x[0, 8, 9] == 1, idx="Top left square should be a 9", coef=coef_for_top_right_corner, priority=3)

### 2.3- Solve the problem

In [62]:
my_solver = SuperSolver(model)
status = my_solver.Solve()
status == Status.OPTIMAL

True

### 2.4- Print the result

In [70]:
def write_result(solver, x):
    size_sudoku = 9
    tab_val = dict()
    for i in range(size_sudoku):
        for j in range(size_sudoku):
            tab_val[i, j] = '.'
            for v in range(1, size_sudoku+1):
                if solver.Value(x[i, j, v]):
                    tab_val[i, j] = v
                    break
    n = size_sudoku
    k = int(sqrt(n))
    for i in range(n):
        for j in range(n):
            if tab_val[i, j]:
                print(tab_val[i, j], end=" ")
            else:
                print(".", end=" ")
            if j + 1 < n and (j + 1) % k == 0:
                print("|", end=" ")
        print()
        if i + 1 < n and (i + 1) % k == 0:
            print("- " * (n + 2))

In [71]:
write_result(my_solver, x)

1 7 4 | 8 2 5 | 3 6 9 
5 9 6 | 1 7 3 | 4 2 8 
3 2 8 | 4 9 6 | 1 7 5 
- - - - - - - - - - - 
6 1 7 | 9 4 2 | 5 8 3 
8 5 3 | 6 1 7 | 9 4 2 
9 4 2 | 3 5 8 | 6 1 7 
- - - - - - - - - - - 
7 6 1 | 2 3 9 | 8 5 4 
4 3 5 | 7 8 1 | 2 9 6 
2 8 9 | 5 6 4 | 7 3 1 


### 2.5- Get local explanations

Ortools_explain enables you to explain why some variables were set to their value in the current solution.

#### Explain why there is a 1 in the top left corner

In [69]:
print(my_solver.ExplainValueOfVar(x[0, 0, 1])['outcome'])

as_optimal


This means that it is possible to find a solution that is as optimal as the previous one where there is no 1 on the top left corner. In this case, the module also returns the list of variables that have changed in the new solution:

In [68]:
print(my_solver.ExplainValueOfVar(x[0, 0, 1])['changed_variables'][:10])

[{'name': 'x_0_0_1', 'old_value': 1, 'new_value': 0}, {'name': 'x_0_0_8', 'old_value': 0, 'new_value': 1}, {'name': 'x_0_1_6', 'old_value': 0, 'new_value': 1}, {'name': 'x_0_1_7', 'old_value': 1, 'new_value': 0}, {'name': 'x_0_3_1', 'old_value': 0, 'new_value': 1}, {'name': 'x_0_3_3', 'old_value': 1, 'new_value': 0}, {'name': 'x_0_4_4', 'old_value': 1, 'new_value': 0}, {'name': 'x_0_4_5', 'old_value': 0, 'new_value': 1}, {'name': 'x_0_5_4', 'old_value': 0, 'new_value': 1}, {'name': 'x_0_5_6', 'old_value': 1, 'new_value': 0}]


#### Explain why there is a 9 in the top right corner

In [73]:
print(my_solver.ExplainValueOfVar(x[0, 8, 9]))

{'outcome': 'less_optimal', 'optimization_scores': {'old_value': [3, 54, -32], 'new_value': [3, 54, -54]}, 'objective_values': [{'id': 'Numbers of 1 on first diagonal', 'old_value': 3, 'new_value': 3}, {'id': 'Maximize sum of first diagonal', 'old_value': 54, 'new_value': 54}, {'id': 'Minimize sum of second diagonal', 'old_value': 72, 'new_value': 54}, {'id': 'Top left square should be a 9', 'old_value': 1, 'new_value': 0}]}


This means that removing the 9 from this square leads to a lower optimisation value at rank 3.

#### Explain why the sum of the first diagonal is higher than the sum of the second diagonal

In [75]:
explanation = my_solver.ExplainWhyNot(sum(v * x[i, i, v] for i in range(size_sudoku) for v in range(1, size_sudoku + 1)) <=
                                              sum(v * x[i, size_sudoku - 1 - i, v] for i in range(size_sudoku) for v in range(1, size_sudoku + 1)))
print(explanation)

{'outcome': 'less_optimal', 'optimization_scores': {'old_value': [3, 54, -32], 'new_value': [3, 54, -122]}, 'objective_values': [{'id': 'Numbers of 1 on first diagonal', 'old_value': 3, 'new_value': 3}, {'id': 'Maximize sum of first diagonal', 'old_value': 54, 'new_value': 54}, {'id': 'Minimize sum of second diagonal', 'old_value': 72, 'new_value': 162}, {'id': 'Top left square should be a 9', 'old_value': 1, 'new_value': 1}]}


#### Explain why the sum of the first line is 45

In [76]:
print(my_solver.ExplainWhyNot(sum(v * x[0, j, v] for j in range(size_sudoku) for v in range(1, size_sudoku + 1)) != 45))

{'outcome': 'infeasible'}


This means that if we force the sum of the first line not to be 45, the model becomes infeasible.